In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import yfinance as yf
from arch import arch_model
from scipy.optimize import minimize

# -------------------------------
# Helper functions
# -------------------------------
def download_prices(tickers, start_date="2018-01-01", end_date=None):
    if end_date is None:
        end_date = dt.date.today().isoformat()
    data = yf.download(tickers, start=start_date, end=end_date, progress=False, auto_adjust=True)["Close"]
    if isinstance(data, pd.Series):
        data = data.to_frame()
    return data.dropna(how="all")

def compute_log_returns(prices):
    return np.log(prices).diff().dropna()

def fit_garch_and_forecast_var(returns, p=1, q=1, forecast_horizon=1):
    forecast_vars = {}
    for col in returns.columns:
        series = returns[col].dropna() * 100  # scale for stability
        try:
            am = arch_model(series, mean='Constant', vol='GARCH', p=p, q=q, dist='normal')
            res = am.fit(disp='off')
            fc = res.forecast(horizon=forecast_horizon, reindex=False)
            var_forecast = fc.variance.values[-1, -1] / (100**2)  # scale back
            forecast_vars[col] = var_forecast
        except:
            forecast_vars[col] = series.var() / (100**2)
    return forecast_vars

def build_cov_matrix(forecast_vars, returns, corr_window=252):
    tickers = list(forecast_vars.keys())
    recent = returns.tail(corr_window)
    corr = recent.corr()
    cov = pd.DataFrame(index=tickers, columns=tickers, dtype=float)
    for i in tickers:
        for j in tickers:
            cov.loc[i, j] = np.sqrt(forecast_vars[i]) * np.sqrt(forecast_vars[j]) * corr.loc[i, j]
    return cov

def estimate_expected_returns(returns, method='ewma', span=60):
    if method == 'ewma':
        return returns.ewm(span=span).mean().iloc[-1]
    elif method == 'mean':
        return returns.mean()
    else:
        raise ValueError('Unknown method')

def max_sharpe_weights(expected_returns, cov_matrix, risk_free_rate=0.0, bounds=(0.0, 1.0)):
    tickers = expected_returns.index.tolist()
    mu = expected_returns.values
    cov = cov_matrix.values
    
    def neg_sharpe(w):
        port_ret = np.dot(w, mu)
        port_vol = np.sqrt(w @ cov @ w)
        return -(port_ret - risk_free_rate) / port_vol if port_vol != 0 else 1e6

    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = tuple([bounds] * len(tickers))
    x0 = np.repeat(1.0 / len(tickers), len(tickers))
    
    res = minimize(neg_sharpe, x0=x0, bounds=bounds, constraints=constraints, method='SLSQP')
    return pd.Series(res.x, index=tickers)

# -------------------------------
# Test for a week ago
# -------------------------------
tickers = ['NVDA', 'PLTR', 'TSLA']
end_date = (dt.date.today() - dt.timedelta(days=7)).isoformat()
start_date = "2020-01-01"

prices = download_prices(tickers, start_date=start_date, end_date=end_date)
returns = compute_log_returns(prices)

forecast_vars = fit_garch_and_forecast_var(returns)
cov_matrix = build_cov_matrix(forecast_vars, returns)
expected_returns = estimate_expected_returns(returns)
weights = max_sharpe_weights(expected_returns, cov_matrix)

print("Portfolio Weights (a week ago):")
print(weights)
print("\nExpected Returns (EWMA):")
print(expected_returns)
print("\nForecast Covariance Matrix:")
print(cov_matrix)


Portfolio Weights (a week ago):
NVDA    4.654103e-01
PLTR    1.110223e-16
TSLA    5.345897e-01
dtype: float64

Expected Returns (EWMA):
Ticker
NVDA    0.000661
PLTR   -0.000158
TSLA    0.000931
Name: 2025-12-31 00:00:00, dtype: float64

Forecast Covariance Matrix:
          NVDA      PLTR      TSLA
NVDA  0.000622  0.000534  0.000396
PLTR  0.000534  0.001421  0.000631
TSLA  0.000396  0.000631  0.000960
